In [2]:
import shutil, os
from pathlib import Path
import pandas as pd
import numpy as np
home_folder = "/kaggle/working/"
new_test_fold = home_folder + "test/"
old_test_fold = '/kaggle/input/image-matching-challenge-2024/test/'
if os.path.exists(new_test_fold):
    shutil.rmtree(new_test_fold)
os.makedirs(new_test_fold)
list_scene_test = [p for p in Path(old_test_fold).iterdir() if p.is_dir()]

user_df = pd.read_csv('/kaggle/input/image-matching-challenge-2024/sample_submission.csv')
orin_image_path = user_df['image_path'].to_list()
print("len sbmission: ", len(user_df))
for scene in list_scene_test:
    scene = str(scene)
    scene_train = os.path.join("/kaggle/input/custom-dataset/train/train/", os.path.basename(scene))
    save_new_test = os.path.join(new_test_fold, os.path.basename(scene), "images")
    if os.path.exists(scene_train):
        os.makedirs(save_new_test, exist_ok = True)
        images_training = os.listdir(scene_train+"/images")
        images_test = os.listdir(scene+"/images")
        num_add = min(50, len(images_test))
        add_imgs = np.random.choice(images_training, num_add)
    #     add_imgs = np.random.choice(images_training, 2)

        for images in add_imgs:
            src = os.path.join(scene_train, "images", images)
            dst = os.path.join(save_new_test, images)
            image_path = dst.replace(home_folder, "")
            if image_path in user_df['image_path'].to_list():
                continue
            try:
                shutil.copy(src, dst)
                new_row = pd.Series({'image_path': image_path, 'dataset': os.path.basename(scene), 'scene': os.path.basename(scene), "rotation_matrix":"", "translation_vector":""})
                user_df.loc[len(user_df)] = new_row
            except:
                print("cannot create new")
print("len after", len(user_df))
for dirpath, dirnames, filenames in os.walk(new_test_fold):
    # Skip the root directory
    if dirpath == new_test_fold:
        continue

    num_files = len(filenames)
    print(f"Folder: {os.path.relpath(dirpath, new_test_fold)} - Number of files: {num_files}")
user_df.to_csv("/kaggle/working/sample_submission.csv", index=False, index_label=False)

PermissionError: [Errno 13] Permission denied: '/kaggle'

In [2]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.3
    Uninstalling kornia_rs-0.1.3:
      Successfully uninstalled kornia_rs-0.1.3


In [ ]:
# General utilities
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

# CV/MLe
import cv2
import torch
from torch import Tensor as T
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import torch
from lightglue import match_pair
from lightglue import LightGlue, ALIKED
from lightglue.utils import load_image, rbd

# 3D reconstruction
import pycolmap

# Data importing into colmap
import sys

# Provided by organizers
# import sys
# sys.path.append("/kaggle/input/custom-dataset")
from database import *
from h5_to_db import *


def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

def load_torch_image(file_name, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

device = K.utils.get_cuda_device_if_available(0)
print(device)
list_scene_test = [p for p in Path("/kaggle/input/image-matching-challenge-2024/test/").iterdir() if p.is_dir()]
DEBUG = len(list_scene_test) == 1
print("DEBUG:", DEBUG)
print(list_scene_test)

In [4]:
def embed_images(
    paths: list[Path],
    model_name: str,
    device: torch.device = torch.device("cpu"),
) -> T:
    """Computes image embeddings.
    
    Returns a tensor of shape [len(filenames), output_dim]
    """
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).eval().to(device)
    
    embeddings = []
    
    for i, path in tqdm(enumerate(paths), desc="Global descriptors"):
        image = load_torch_image(path)
        
        with torch.inference_mode():
            inputs = processor(images=image, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs) # last_hidden_state and pooled
            
            # Max pooling over all the hidden states but the first (starting token)
            # To obtain a tensor of shape [1, output_dim]
            # We normalize so that distances are computed in a better fashion later
            embedding = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=-1, p=2)
            
        embeddings.append(embedding.detach().cpu())
    return torch.cat(embeddings, dim=0)
def get_pairs_exhaustive(lst: list[Any]) -> list[tuple[int, int]]:
    """Obtains all possible index pairs of a list"""
    return list(itertools.combinations(range(len(lst)), 2))            
    
def get_image_pairs(
    paths: list[Path],
    model_name: str,
    similarity_threshold: float = 0.6,
    tolerance: int = 1000,
    min_matches: int = 20,
    exhaustive_if_less: int = 20,
    p: float = 2.0,
    device: torch.device = torch.device("cpu"),
) -> list[tuple[int, int]]:
    """Obtains pairs of similar images"""
    if len(paths) <= exhaustive_if_less:
        return get_pairs_exhaustive(paths)
    
    matches = []
    
    # Embed images and compute distances for filtering
    embeddings = embed_images(paths, model_name, device)
    distances = torch.cdist(embeddings, embeddings, p=p)
    
    # Remove pairs above similarity threshold (if enough)
    mask = distances <= similarity_threshold
    image_indices = np.arange(len(paths))
    
    for current_image_index in range(len(paths)):
        mask_row = mask[current_image_index]
        indices_to_match = image_indices[mask_row]
        
        # We don't have enough matches below the threshold, we pick most similar ones
        if len(indices_to_match) < min_matches:
            indices_to_match = np.argsort(distances[current_image_index])[:min_matches]
            
        for other_image_index in indices_to_match:
            # Skip an image matching itself
            if other_image_index == current_image_index:
                continue
            
            # We need to check if we are below a certain distance tolerance 
            # since for images that don't have enough matches, we picked
            # the most similar ones (which could all still be very different 
            # to the image we are analyzing)
            if distances[current_image_index, other_image_index] < tolerance:
                # Add the pair in a sorted manner to avoid redundancy
                matches.append(tuple(sorted((current_image_index, other_image_index.item()))))
                
    return sorted(list(set(matches)))

In [5]:
def detect_keypoints(
    paths: list[Path],
    feature_dir: Path,
    num_features: int = 4096,
    resize_to: int = 1024,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Detects the keypoints in a list of images with ALIKED
    
    Stores them in feature_dir/keypoints.h5 and feature_dir/descriptors.h5
    to be used later with LightGlue
    """
    dtype = torch.float32 # ALIKED has issues with float16
    
    extractor = ALIKED(
        max_num_keypoints=num_features, 
        detection_threshold=0.01, 
        resize=resize_to
    ).eval().to(device, dtype)
    
    feature_dir.mkdir(parents=True, exist_ok=True)
    print("computing detect_keypoints", len(paths))
    with h5py.File(feature_dir / "keypoints.h5", mode="w") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="w") as f_descriptors:
        
        for path in tqdm(paths, desc="detect_keypoints"):
#         for path in paths:
            
            key = path.name
            
            with torch.inference_mode():
                image = load_torch_image(path, device=device).to(dtype)
                features = extractor.extract(image)
                
                f_keypoints[key] = features["keypoints"].squeeze().detach().cpu().numpy()
                f_descriptors[key] = features["descriptors"].squeeze().detach().cpu().numpy()

In [6]:
def keypoint_distances(
    paths: list[Path],
    index_pairs: list[tuple[int, int]],
    feature_dir: Path,
    min_matches: int = 15,
    verbose: bool = True,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Computes distances between keypoints of images.
    
    Stores output at feature_dir/matches.h5
    """
    
    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)
    print("computing keypoint_distances", len(paths), "len pair", len(index_pairs))
    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors, \
         h5py.File(feature_dir / "matches.h5", mode="w") as f_matches:
        
            for idx1, idx2 in tqdm(index_pairs, desc="keypoint_distances"):
#             for idx1, idx2 in index_pairs:
                key1, key2 = paths[idx1].name, paths[idx2].name

                keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
                keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
                descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
                descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)

                with torch.inference_mode():
                    distances, indices = matcher(
                        descriptors1, 
                        descriptors2, 
                        KF.laf_from_center_scale_ori(keypoints1[None]),
                        KF.laf_from_center_scale_ori(keypoints2[None]),
                    )

                # We have matches to consider
                n_matches = len(indices)
                if n_matches:
                    if verbose:
                        print(f"{key1}-{key2}: {n_matches} matches")
                    # Store the matches in the group of one image
                    if n_matches >= min_matches:
                        group  = f_matches.require_group(key1)
                        group.create_dataset(key2, data=indices.detach().cpu().numpy().reshape(-1, 2))

In [7]:
# if DEBUG:
#     images_list = list(Path("/kaggle/input/image-matching-challenge-2024/test/church/images/").glob("*.png"))[:10]
#     index_pairs = get_image_pairs(images_list, "dinov2")
#     print(index_pairs)
#     feature_dir = Path("./sample_test_features")
#     detect_keypoints(images_list, feature_dir)
#     keypoint_distances(images_list, index_pairs, feature_dir, verbose=False, device=device)
#     for idx1, idx2 in index_pairs:
#         key1, key2 = images_list[idx1], images_list[idx2]
#         img1 = load_torch_image(key1)
#         img2 = load_torch_image(key2)
#         fig, ax = plt.subplots(1, 2, figsize=(10, 20))
#         ax[0].imshow(img1[0, ...].permute(1,2,0).cpu())
#         ax[1].imshow(img2[0, ...].permute(1,2,0).cpu())
#         plt.show()

In [8]:
def import_into_colmap(
    path: Path,
    feature_dir: Path,
    database_path: str = "colmap.db",
) -> None:
    """Adds keypoints into colmap"""
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    path1 = path
    path2 = Path(str(path1).replace("input/image-matching-challenge-2024", "working"))
    orin_image_name = os.listdir(path1)
    fname_to_id = add_keypoints(db, feature_dir, path1, path2, orin_image_name, "", "simple-pinhole", single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()

In [9]:
class Config:
    base_path: Path = Path("/kaggle/input/image-matching-challenge-2024/")
#     base_path: Path = Path("/kaggle/input/custom-dataset")
        
    feature_dir: Path = Path.cwd() / ".feature_outputs"
        
    device: torch.device = K.utils.get_cuda_device_if_available(0)
    
    pair_matching_args = {
        "model_name": "/kaggle/input/dinov2/pytorch/large/1",
        "similarity_threshold": 0.6,
        "tolerance": 500,
        "min_matches": 50,
        "exhaustive_if_less": 50,
        "p": 2.0,
    }
    
    keypoint_detection_args = {
        "num_features": 4096,
        "resize_to": 1024,
    }
    
    keypoint_distances_args = {
        "min_matches": 15,
        "verbose": False,
    }
    
    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
    }

In [10]:
def parse_sample_submission() -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as 
    
    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open("sample_submission.csv", "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                if image_path in orin_image_path:
                    base_path = Path("/kaggle/input/image-matching-challenge-2024/")
                else:
                    base_path = Path("/kaggle/working/")
                data_dict[dataset][scene].append(Path(base_path / image_path))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict
def create_submission(
    results: dict,
    data_dict: dict[dict[str, list[Path]]],
    base_path: Path,
    orin_image_path
) -> None:
    """Prepares a submission file."""
    
    with open("submission.csv", "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")
        
        for dataset in data_dict:
            # Only write results for datasets with images that have results 
            if dataset in results:
                res = results[dataset]
            else:
                res = {}
            
            # Same for scenes
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                scene_res_2 = {}
                for skey in scene_res.keys():
                    idx = str(skey).find("test")
                
                    skey2 = str(skey.relative_to(base_path))
                    scene_res_2[skey2] = scene_res[skey]
                    
                # Write the row with rotation and translation matrices
                for image in data_dict[dataset][scene]:
                    image = str(image)
                    idx = image.find("test")
                    image = image[idx:]
                    if image in scene_res_2:
                        print(image)
                        R = scene_res_2[image]["R"].reshape(-1)
                        T = scene_res_2[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    if image in orin_image_path:
                        f.write(f"{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n")

In [11]:
# DEBUG = False

In [12]:
results = {}

data_dict = parse_sample_submission()
datasets = list(data_dict.keys())

for dataset in datasets:
    if dataset not in results:
        results[dataset] = {}

    for scene in data_dict[dataset]:
        images_dir = data_dict[dataset][scene][0].parent
        results[dataset][scene] = {}
        image_paths = data_dict[dataset][scene]
        print (f"{scene}: Got {len(image_paths)} images")

        try:
            feature_dir = Config.feature_dir / f"{dataset}_{scene}"
            feature_dir.mkdir(parents=True, exist_ok=True)
            database_path = feature_dir / "colmap.db"
            if database_path.exists():
                database_path.unlink()

            # 1. Get the pairs of images that are somewhat similar
            index_pairs = get_image_pairs(
                image_paths,
                **Config.pair_matching_args,
                device=Config.device,
            )
            gc.collect()

            # 2. Detect keypoints of all images
            detect_keypoints(
                image_paths,
                feature_dir,
                **Config.keypoint_detection_args,
                device=device,
            )
            gc.collect()

            # 3. Match  keypoints of pairs of similar images
            keypoint_distances(
                image_paths, 
                index_pairs, 
                feature_dir,
                **Config.keypoint_distances_args,
                device=device,
            )
            gc.collect()

            sleep(1)

            # 4.1. Import keypoint distances of matches into colmap for RANSAC 
            import_into_colmap(
                images_dir, 
                feature_dir, 
                database_path,
            )

            output_path = feature_dir / "colmap_rec_aliked"
            output_path.mkdir(parents=True, exist_ok=True)

            # 4.2. Compute RANSAC (detect match outliers)
            # By doing it exhaustively we guarantee we will find the best possible Configuration
            pycolmap.match_exhaustive(database_path)

            mapper_options = pycolmap.IncrementalPipelineOptions(**Config.colmap_mapper_options)

            # 5.1 Incrementally start reconstructing the scene (sparse reconstruction)
            # The process starts from a random pair of images and is incrementally extended by 
            # registering new images and triangulating new points.
            maps = pycolmap.incremental_mapping(
                database_path=database_path, 
                image_path=images_dir,
                output_path=output_path, 
                options=mapper_options,
            )

#             print(maps)
            clear_output(wait=False)

            # 5.2. Look for the best reconstruction: The incremental mapping offered by 
            # pycolmap attempts to reconstruct multiple models, we must pick the best one
            images_registered  = 0
            best_idx = None

            print ("Looking for the best reconstruction")

            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print(idx1, rec.summary())
                    try:
                        if len(rec.images) > images_registered:
                            images_registered = len(rec.images)
                            best_idx = idx1
                    except Exception:
                        continue

            # Parse the reconstruction object to get the rotation matrix and translation vector
            # obtained for each image in the reconstruction
            if best_idx is not None:
                for k, im in maps[best_idx].images.items():
                    key = Config.base_path / "test" / scene / "images" / im.name
                    results[dataset][scene][key] = {}
                    results[dataset][scene][key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                    results[dataset][scene][key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

            print(f"Registered: {dataset} / {scene} -> {len(results[dataset][scene])} images")
            print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
            create_submission(results, data_dict, Config.base_path, orin_image_path)
            gc.collect()

        except Exception as e:
            print(e)

Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 65
	num_cameras = 65
	num_points3D = 19032
	num_observations = 119394
	mean_track_length = 6.27333
	mean_observations_per_image = 1836.83
	mean_reprojection_error = 0.959565
Registered: church / church -> 65 images
Total: church / church -> 65 images
test/church/images/00046.png
test/church/images/00090.png
test/church/images/00092.png
test/church/images/00087.png
test/church/images/00050.png
test/church/images/00068.png
test/church/images/00083.png
test/church/images/00096.png
test/church/images/00069.png
test/church/images/00081.png
test/church/images/00042.png
test/church/images/00018.png
test/church/images/00030.png
test/church/images/00024.png
test/church/images/00032.png
test/church/images/00026.png
test/church/images/00037.png
test/church/images/00008.png
test/church/images/00035.png
test/church/images/00021.png
test/church/images/00010.png
test/church/images/00039.png
test/church/images/00011.png
test/churc

# Evaluate

In [13]:
import time
import math
import numpy as np
import pandas as pd
import pandas.api.types

_EPS = np.finfo(float).eps * 4.0

# mAA evaluation thresholds per scene, different accoring to the scene
translation_thresholds_meters_dict = {
 'multi-temporal-temple-baalshamin':  np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'pond':                              np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'transp_obj_glass_cylinder':         np.array([0.0025, 0.005, 0.01, 0.02, 0.05, 0.1]),
 'transp_obj_glass_cup':              np.array([0.0025, 0.005, 0.01, 0.02, 0.05, 0.1]),
 'church':                            np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'lizard':                            np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]),
 'dioscuri':                          np.array([0.025,  0.05,  0.1,  0.2,  0.5,  1.0]), 
}


def vector_norm(data, axis=None, out=None):
    '''Return length, i.e. Euclidean norm, of ndarray along axis.'''
    data = np.array(data, dtype=np.float64, copy=True)
    if out is None:
        if data.ndim == 1:
            return math.sqrt(np.dot(data, data))
        data *= data
        out = np.atleast_1d(np.sum(data, axis=axis))
        np.sqrt(out, out)
        return out
    data *= data
    np.sum(data, axis=axis, out=out)
    np.sqrt(out, out)
    return None


def quaternion_matrix(quaternion):
    '''Return homogeneous rotation matrix from quaternion.'''
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    if n < _EPS:
        # print("special case")
        return np.identity(4)
    q *= math.sqrt(2.0 / n)
    q = np.outer(q, q)
    return np.array(
        [
            [
                1.0 - q[2, 2] - q[3, 3],
                q[1, 2] - q[3, 0],
                q[1, 3] + q[2, 0],
                0.0,
            ],
            [
                q[1, 2] + q[3, 0],
                1.0 - q[1, 1] - q[3, 3],
                q[2, 3] - q[1, 0],
                0.0,
            ],
            [
                q[1, 3] - q[2, 0],
                q[2, 3] + q[1, 0],
                1.0 - q[1, 1] - q[2, 2],
                0.0,
            ],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )


# based on the 3D registration from https://github.com/cgohlke/transformations
def affine_matrix_from_points(v0, v1, shear=False, scale=True, usesvd=True):
    '''Return affine transform matrix to register two point sets.
    v0 and v1 are shape (ndims, -1) arrays of at least ndims non-homogeneous
    coordinates, where ndims is the dimensionality of the coordinate space.
    If shear is False, a similarity transformation matrix is returned.
    If also scale is False, a rigid/Euclidean traffansformation matrix
    is returned.
    By default the algorithm by Hartley and Zissermann [15] is used.
    If usesvd is True, similarity and Euclidean transformation matrices
    are calculated by minimizing the weighted sum of squared deviations
    (RMSD) according to the algorithm by Kabsch [8].
    Otherwise, and if ndims is 3, the quaternion based algorithm by Horn [9]
    is used, which is slower when using this Python implementation.
    The returned matrix performs rotation, translation and uniform scaling
    (if specified).'''
    
    v0 = np.array(v0, dtype=np.float64, copy=True)
    v1 = np.array(v1, dtype=np.float64, copy=True)

    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")

    # move centroids to origin
    t0 = -np.mean(v0, axis=1)
    M0 = np.identity(ndims + 1)
    M0[:ndims, ndims] = t0
    v0 += t0.reshape(ndims, 1)
    t1 = -np.mean(v1, axis=1)
    M1 = np.identity(ndims + 1)
    M1[:ndims, ndims] = t1
    v1 += t1.reshape(ndims, 1)

    if shear:
        # Affine transformation
        A = np.concatenate((v0, v1), axis=0)
        u, s, vh = np.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]
        C = vh[ndims: 2 * ndims]
        t = np.dot(C, np.linalg.pinv(B))
        t = np.concatenate((t, np.zeros((ndims, 1))), axis=1)
        M = np.vstack((t, ((0.0,) * ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        # Rigid transformation via SVD of covariance matrix
        u, s, vh = np.linalg.svd(np.dot(v1, v0.T))
        # rotation matrix from SVD orthonormal bases
        R = np.dot(u, vh)
        if np.linalg.det(R) < 0.0:
            # R does not constitute right handed system
            R -= np.outer(u[:, ndims - 1], vh[ndims - 1, :] * 2.0)
            s[-1] *= -1.0
        # homogeneous transformation matrix
        M = np.identity(ndims + 1)
        M[:ndims, :ndims] = R
    else:
        # Rigid transformation matrix via quaternion
        # compute symmetric matrix N
        xx, yy, zz = np.sum(v0 * v1, axis=1)
        xy, yz, zx = np.sum(v0 * np.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = np.sum(v0 * np.roll(v1, -2, axis=0), axis=1)
        N = [
            [xx + yy + zz, 0.0, 0.0, 0.0],
            [yz - zy, xx - yy - zz, 0.0, 0.0],
            [zx - xz, xy + yx, yy - xx - zz, 0.0],
            [xy - yx, zx + xz, yz + zy, zz - xx - yy],
        ]
        # quaternion: eigenvector corresponding to most positive eigenvalue
        w, V = np.linalg.eigh(N)
        q = V[:, np.argmax(w)]
        # print (vector_norm(q), np.linalg.norm(q))
        q /= vector_norm(q)  # unit quaternion
        # homogeneous transformation matrix
        M = quaternion_matrix(q)

    if scale and not shear:
        # Affine transformation; scale is ratio of RMS deviations from centroid
        v0 *= v0
        v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))

    # move centroids back
    M = np.dot(np.linalg.inv(M1), np.dot(M, M0))
    M /= M[ndims, ndims]

    # print("transformation matrix Python Script: ", M)

    return M


# This is the IMC 3D error metric code
def register_by_Horn(ev_coord, gt_coord, ransac_threshold, inl_cf, strict_cf):
    '''Return the best similarity transforms T that registers 3D points pt_ev in <ev_coord> to
    the corresponding ones pt_gt in <gt_coord> according to a RANSAC-like approach for each
    threshold value th in <ransac_threshold>.
    
    Given th, each triplet of 3D correspondences is examined if not already present as strict inlier,
    a correspondence is a strict inlier if <strict_cf> * err_best < th, where err_best is the registration
    error for the best model so far.
    The minimal model given by the triplet is then refined using also its inliers if their total is greater
    than <inl_cf> * ninl_best, where ninl_best is th number of inliers for the best model so far. Inliers
    are 3D correspondences (pt_ev, pt_gt) for which the Euclidean distance |pt_gt-T*pt_ev| is less than th.'''
    
    # remove invalid cameras, the index is returned
    idx_cams = np.all(np.isfinite(ev_coord), axis=0)
    ev_coord = ev_coord[:, idx_cams]
    gt_coord = gt_coord[:, idx_cams]

    # initialization
    n = ev_coord.shape[1]
    r = ransac_threshold.shape[0]
    ransac_threshold = np.expand_dims(ransac_threshold, axis=0)
    ransac_threshold2 = ransac_threshold**2
    ev_coord_1 = np.vstack((ev_coord, np.ones(n)))

    max_no_inl = np.zeros((1, r))
    best_inl_err = np.full(r, np.Inf)
    best_transf_matrix = np.zeros((r, 4, 4))
    best_err = np.full((n, r), np.Inf)
    strict_inl = np.full((n, r), False)
    triplets_used = np.zeros((3, r))

    # run on camera triplets
    for ii in range(n-2):
        for jj in range(ii+1, n-1):
            for kk in range(jj+1, n):
                i = [ii, jj, kk]
                triplets_used_now = np.full((n), False)
                triplets_used_now[i] = True
                # if both ii, jj, kk are strict inliers for the best current model just skip
                if np.all(strict_inl[i]):
                    continue
                # get transformation T by Horn on the triplet camera center correspondences
                transf_matrix = affine_matrix_from_points(ev_coord[:, i], gt_coord[:, i], usesvd=False)
                # apply transformation T to test camera centres
                rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                # compute error and inliers
                err = np.sum((rotranslated - gt_coord)**2, axis=0)
                inl = np.expand_dims(err, axis=1) < ransac_threshold2
                no_inl = np.sum(inl, axis=0)
                # if the number of inliers is close to that of the best model so far, go for refinement
                to_ref = np.squeeze(((no_inl > 2) & (no_inl > max_no_inl * inl_cf)), axis=0)
                for q in np.argwhere(to_ref):                        
                    qq = q[0]
                    if np.any(np.all((np.expand_dims(inl[:, qq], axis=1) == inl[:, :qq]), axis=0)):
                        # already done for this set of inliers
                        continue
                    # get transformation T by Horn on the inlier camera center correspondences
                    transf_matrix = affine_matrix_from_points(ev_coord[:, inl[:, qq]], gt_coord[:, inl[:, qq]])
                    # apply transformation T to test camera centres
                    rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                    # compute error and inliers
                    err_ref = np.sum((rotranslated - gt_coord)**2, axis=0)
                    err_ref_sum = np.sum(err_ref, axis=0)
                    err_ref = np.expand_dims(err_ref, axis=1)
                    inl_ref = err_ref < ransac_threshold2
                    no_inl_ref = np.sum(inl_ref, axis=0)
                    # update the model if better for each threshold
                    to_update = np.squeeze((no_inl_ref > max_no_inl) | ((no_inl_ref == max_no_inl) & (err_ref_sum < best_inl_err)), axis=0)
                    if np.any(to_update):
                        triplets_used[0, to_update] = ii
                        triplets_used[1, to_update] = jj
                        triplets_used[2, to_update] = kk
                        max_no_inl[:, to_update] = no_inl_ref[to_update]
                        best_err[:, to_update] = np.sqrt(err_ref)
                        best_inl_err[to_update] = err_ref_sum
                        strict_inl[:, to_update] = (best_err[:, to_update] < strict_cf * ransac_threshold[:, to_update])
                        best_transf_matrix[to_update] = transf_matrix

    for i in range(r):
       print(f'Registered cameras {int(max_no_inl[0, i])}/{n} for threshold {ransac_threshold[0, i]}')

    best_model = {
        "valid_cams": idx_cams,        
        "no_inl": max_no_inl,
        "err": best_err,
        "triplets_used": triplets_used,
        "transf_matrix": best_transf_matrix}
    return best_model


# mAA computation
def mAA_on_cameras(err, thresholds, n, skip_top_thresholds, to_dec=3):
    '''mAA is the mean of mAA_i, where for each threshold th_i in <thresholds>, excluding the first <skip_top_thresholds values>,
    mAA_i = max(0, sum(err_i < th_i) - <to_dec>) / (n - <to_dec>)
    where <n> is the number of ground-truth cameras and err_i is the camera registration error for the best 
    registration corresponding to threshold th_i'''
    
    aux = err[:, skip_top_thresholds:] < np.expand_dims(np.asarray(thresholds[skip_top_thresholds:]), axis=0)
    return np.sum(np.maximum(np.sum(aux, axis=0) - to_dec, 0)) / (len(thresholds[skip_top_thresholds:]) * (n - to_dec))


# import data - no error handling in case float(x) fails
def get_camera_centers_from_df(df):
    out = {}
    for row in df.iterrows():
        row = row[1]
        fname = row['image_path']
        R = np.array([float(x) for x in (row['rotation_matrix'].split(';'))]).reshape(3,3)
        t = np.array([float(x) for x in (row['translation_vector'].split(';'))]).reshape(3)
        center = -R.T @ t
        out[fname] = center
    return out


def evaluate_rec(gt_df, user_df, inl_cf = 0.8, strict_cf=0.5, skip_top_thresholds=2, to_dec=3,
                 thresholds=[0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2]):
    ''' Register the <user_df> camera centers to the ground-truth <gt_df> camera centers and
    return the corresponding mAA as the average percentage of registered camera threshold.
    
    For each threshold value in <thresholds>, the best similarity transformation found which
    maximizes the number of registered cameras is employed. A camera is marked as registered
    if after the transformation its Euclidean distance to the corresponding ground-truth camera
    center is less than the mentioned threshold. Current measurements are in meter.
    
    Registration parameters:
    <inl_cf> coefficient to activate registration refinement, set to 1 to refine a new model
    only when it gives more inliers, to 0 to refine a new model always; high values increase
    speed but decrease precision.
    <strict_cf> threshold coefficient to define strict inliers for the best registration so far,
    new minimal models made up of strict inliers are skipped. It can vary from 0 (slower) to
    1 (faster); set to -1 to check exhaustively all the minimal model triplets.

    mAA parameters:
    <skip_top_thresholds> excluded lower thresholds in the mAA computation; in case of using
    heuristics for the registration, i.e. inl_cf!=0 and strict_cf!=-1, best model for lower
    threshold can be not the optimal, so skip them in the mAA computation.
    <to_dec> excludes the minimal model cameras from the computation of the mAA. Given the
    minimal model, i.e. three pairs of 3D correspondences, there is a high chance to register by
    a similarity transformation at any threshold, so do not account for mAA'''
    
    # get camera centers
    ucameras = get_camera_centers_from_df(user_df)
    gcameras = get_camera_centers_from_df(gt_df)    

    # the denominator for mAA ratio
    m = gt_df.shape[0]
    
    # get the image list to use
    good_cams = []
    for image_path in gcameras.keys():
        if image_path in ucameras.keys():
            good_cams.append(image_path)
    print("good_cams", good_cams)
        
    # put corresponding camera centers into matrices
    n = len(good_cams)
    u_cameras = np.zeros((3, n))
    g_cameras = np.zeros((3, n))
    
    ii = 0
    for i in good_cams:
        u_cameras[:, ii] = ucameras[i]
        g_cameras[:, ii] = gcameras[i]
        ii += 1
        
    # Horn camera centers registration, a different best model for each camera threshold
    model = register_by_Horn(u_cameras, g_cameras, np.asarray(thresholds), inl_cf, strict_cf)
    
    # transformation matrix
    print("\nTransformation matrix for maximum threshold")
    T = np.squeeze(model['transf_matrix'][-1])
    print(T)
    
    # mAA
    mAA = mAA_on_cameras(model["err"], thresholds, m, skip_top_thresholds, to_dec)
    # print(f'mAA = {mAA * 100 : .2f}% considering {m} input cameras - {to_dec}')
    return mAA


def score(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    '''The metric is an mean average accuracy between solution and submission camera centers.
    Prior to calculate the metric, a function performs exhaustive registration (like RANSAC, but
    not random, considering all possible configurations) to align the user camera system to the GT'''
    
    scenes = list(set(solution['dataset'].tolist()))
    results_per_dataset = []
    for dataset in scenes:
        print(f"\n*** {dataset} ***")
        start = time.time()
        gt_ds = solution[solution['dataset'] == dataset]
        user_ds = submission[submission['dataset'] == dataset]
        gt_ds = gt_ds.sort_values(by=['image_path'], ascending = True)
        user_ds = user_ds.sort_values(by=['image_path'], ascending = True)
        result = evaluate_rec(gt_ds, user_ds, inl_cf=0, strict_cf=-1, skip_top_thresholds=0, to_dec=3,
                 thresholds=translation_thresholds_meters_dict[dataset])
        end = time.time()
        print(f"\nmAA: {result*100}%")
        print("Running time: %s" % (end - start))        
        results_per_dataset.append(result)
    return float(np.array(results_per_dataset).mean())

In [14]:
import os, time
trans = np.eye(4)
trans [:3, -1] = [0,0,0]

gt_csv = '/kaggle/input/custom-dataset/test_gt.csv'
user_csv = '/kaggle/working/submission.csv'

gt_df = pd.read_csv(gt_csv).sort_values(by='image_path')
user_df = pd.read_csv(user_csv)

user_df['image_path'] = user_df['image_path'].apply(lambda x: os.path.basename(x))
user_df = user_df.head(41)
user_df = user_df.sort_values(by='image_path')



start = time.time()
res = score(gt_df, user_df)
end = time.time()

print(f"\nGlobal mAA: {res*100}%")
print("Total running time: %s" % (end - start))


*** church ***
good_cams ['00001.png', '00006.png', '00008.png', '00010.png', '00011.png', '00012.png', '00013.png', '00018.png', '00021.png', '00024.png', '00026.png', '00029.png', '00030.png', '00032.png', '00035.png', '00037.png', '00039.png', '00042.png', '00046.png', '00050.png', '00058.png', '00059.png', '00060.png', '00061.png', '00063.png', '00066.png', '00068.png', '00069.png', '00072.png', '00074.png', '00076.png', '00081.png', '00083.png', '00087.png', '00090.png', '00092.png', '00096.png', '00098.png', '00102.png', '00104.png', '00111.png']
Registered cameras 4/41 for threshold 0.025
Registered cameras 5/41 for threshold 0.05
Registered cameras 8/41 for threshold 0.1
Registered cameras 13/41 for threshold 0.2
Registered cameras 21/41 for threshold 0.5
Registered cameras 21/41 for threshold 1.0

Transformation matrix for maximum threshold
[[ -4.59711811   3.3485602    8.15487069 -21.08112498]
 [ -3.1639592    7.95759819  -5.05116418  14.158655  ]
 [ -8.22825216  -4.93072429

In [15]:
!ls

__notebook__.ipynb  sample_submission.csv  submission.csv  test
